In [1]:
print("hola")

hola


In [3]:
import numpy as np

In [17]:
line = np.arange(3).astype(np.int16)
c = np.ones_like(line)


In [20]:
def poly_exp_1D(f, c, sigma):
    """
    Calculates the local polynomial expansion of a 1D signal.
    
    $f ~ x^T A x + B^T x + C$
    
    If f[i] and c[i] are the signal value and certainty of pixel i then
    A[i] is a 1x1 array representing the quadratic term of the polynomial, B[i]
    is a 1-element array representing the linear term, and C[i] is a scalar
    representing the constant term.
    
    Parameters
    ----------
    f
        Input signal
    c
        Certainty of signal
    sigma
        Standard deviation of applicability Gaussian kernel
    Returns
    -------
    A
        Quadratic term of polynomial expansion
    B
        Linear term of polynomial expansion
    C
        Constant term of polynomial expansion
    """
    # Kernel applicability
    n = int(4 * sigma + 1)
    x = np.arange(-n, n + 1, dtype=np.int32)
    a = np.exp(-(x**2) / (2 * sigma**2))

    # b: calculate b from the paper.
    b = np.stack([np.ones(a.shape), x, x**2], axis=-1)
    print(b.shape)

    # Pre-calculate product of certainty and signal
    cf = c * f

    # G and v are used to calculate "r" from the paper: v = G*r
    # r is the parametrization of the 2nd order polynomial for f
    G = np.empty(list(f.shape))
    v = np.empty(list(f.shape))
    print("G.shape =", G.shape)
    print("v.shape =", v.shape)

    # Apply cross-correlation

    # Pre-calculate quantities recommended in paper
    ab = np.einsum("i,ij->ij", a, b)
    abb = np.einsum("ij,ik->ijk", ab, b)
    print("a", a)
    print("b", b)
    print("ab", ab)
    print("abb", abb)

    # Calculate G and v for each pixel with cross-correlation
    for j in range(b.shape):
        G[..., j] = scipy.ndimage.correlate1d(
            c, abb[..., j], axis=0, mode="constant", cval=0
        )

    v = scipy.ndimage.correlate1d(
        cf, ab, axis=0, mode="constant", cval=0
    )

    # Solve r for each pixel
    r = np.linalg.solve(G, v)

    # Quadratic term
    A = np.empty(list(f.shape))
    A[..., 0] = r[..., 2]

    # Linear term
    B = np.empty(list(f.shape))
    B[..., 0] = r[..., 1]

    # constant term
    C = r[..., 0]

    # b: [n, n, 6]
    # r: [f, f, 6]
    # f: [f, f]
    # e = b*r - f

    return A, B, C

In [28]:
def poly_exp(f, c, sigma):
    """
    Calculates the local polynomial expansion of a 2D signal, as described by Farneback
    Uses separable normalized correlation
    $f ~ x^T A x + B^T x + C$
    If f[i, j] and c[i, j] are the signal value and certainty of pixel (i, j) then
    A[i, j] is a 2x2 array representing the quadratic term of the polynomial, B[i, j]
    is a 2-element array representing the linear term, and C[i, j] is a scalar
    representing the constant term.
    Parameters
    ----------
    f
        Input signal
    c
        Certainty of signal
    sigma
        Standard deviation of applicability Gaussian kernel
    Returns
    -------
    A
        Quadratic term of polynomial expansion
    B
        Linear term of polynomial expansion
    C
        Constant term of polynomial expansion
    """
    # Calculate applicability kernel (1D because it is separable)
    n = int(4 * sigma + 1)
    x = np.arange(-n, n + 1, dtype=np.int32)
    a = np.exp(-(x**2) / (2 * sigma**2))  # a: applicability kernel [n]

    # b: calculate b from the paper. Calculate separately for X and Y dimensions
    # [n, 6]
    bx = np.stack(
        [np.ones(a.shape), x, np.ones(a.shape), x**2, np.ones(a.shape), x], axis=-1
    )
    by = np.stack(
        [
            np.ones(a.shape),
            np.ones(a.shape),
            x,
            np.ones(a.shape),
            x**2,
            x,
        ],
        axis=-1,
    )

    # Pre-calculate product of certainty and signal
    cf = c * f

    # G and v are used to calculate "r" from the paper: v = G*r
    # r is the parametrization of the 2nd order polynomial for f
    G = np.empty(list(f.shape) + [bx.shape[-1]] * 2)
    v = np.empty(list(f.shape) + [bx.shape[-1]])

    # Apply separable cross-correlations

    # Pre-calculate quantities recommended in paper
    ab = np.einsum("i,ij->ij", a, bx)
    abb = np.einsum("ij,ik->ijk", ab, bx)
    print("a", a, a.shape)
    print("bx", bx, bx.shape)
    print("ab", ab)
    print("abb", abb)
    
    # Calculate G and v for each pixel with cross-correlation
    for i in range(bx.shape[-1]):
        for j in range(bx.shape[-1]):
            G[..., i, j] = scipy.ndimage.correlate1d(
                c, abb[..., i, j], axis=0, mode="constant", cval=0
            )

        v[..., i] = scipy.ndimage.correlate1d(
            cf, ab[..., i], axis=0, mode="constant", cval=0
        )

    # Pre-calculate quantities recommended in paper
    ab = np.einsum("i,ij->ij", a, by)
    abb = np.einsum("ij,ik->ijk", ab, by)

    # Calculate G and v for each pixel with cross-correlation
    for i in range(bx.shape[-1]):
        for j in range(bx.shape[-1]):
            G[..., i, j] = scipy.ndimage.correlate1d(
                G[..., i, j], abb[..., i, j], axis=1, mode="constant", cval=0
            )

        v[..., i] = scipy.ndimage.correlate1d(
            v[..., i], ab[..., i], axis=1, mode="constant", cval=0
        )

    # Solve r for each pixel
    r = np.linalg.solve(G, v)

    # Quadratic term
    A = np.empty(list(f.shape) + [2, 2])
    A[..., 0, 0] = r[..., 3]
    A[..., 0, 1] = r[..., 5] / 2
    A[..., 1, 0] = A[..., 0, 1]
    A[..., 1, 1] = r[..., 4]

    # Linear term
    B = np.empty(list(f.shape) + [2])
    B[..., 0] = r[..., 1]
    B[..., 1] = r[..., 2]

    # constant term
    C = r[..., 0]

    # b: [n, n, 6]
    # r: [f, f, 6]
    # f: [f, f]
    # e = b*r - f

    return A, B, C

In [29]:
poly_exp(line, c, sigma=1.0)

a [3.72665317e-06 3.35462628e-04 1.11089965e-02 1.35335283e-01
 6.06530660e-01 1.00000000e+00 6.06530660e-01 1.35335283e-01
 1.11089965e-02 3.35462628e-04 3.72665317e-06] (11,)
bx [[ 1. -5.  1. 25.  1. -5.]
 [ 1. -4.  1. 16.  1. -4.]
 [ 1. -3.  1.  9.  1. -3.]
 [ 1. -2.  1.  4.  1. -2.]
 [ 1. -1.  1.  1.  1. -1.]
 [ 1.  0.  1.  0.  1.  0.]
 [ 1.  1.  1.  1.  1.  1.]
 [ 1.  2.  1.  4.  1.  2.]
 [ 1.  3.  1.  9.  1.  3.]
 [ 1.  4.  1. 16.  1.  4.]
 [ 1.  5.  1. 25.  1.  5.]] (11, 6)
ab [[ 3.72665317e-06 -1.86332659e-05  3.72665317e-06  9.31663293e-05
   3.72665317e-06 -1.86332659e-05]
 [ 3.35462628e-04 -1.34185051e-03  3.35462628e-04  5.36740205e-03
   3.35462628e-04 -1.34185051e-03]
 [ 1.11089965e-02 -3.33269896e-02  1.11089965e-02  9.99809688e-02
   1.11089965e-02 -3.33269896e-02]
 [ 1.35335283e-01 -2.70670566e-01  1.35335283e-01  5.41341133e-01
   1.35335283e-01 -2.70670566e-01]
 [ 6.06530660e-01 -6.06530660e-01  6.06530660e-01  6.06530660e-01
   6.06530660e-01 -6.06530660e-01]
 [ 1.0

NameError: name 'scipy' is not defined

In [21]:
poly_exp_1D(line, c, sigma=1.0)

(11, 3)
G.shape = (3,)
v.shape = (3,)
a [3.72665317e-06 3.35462628e-04 1.11089965e-02 1.35335283e-01
 6.06530660e-01 1.00000000e+00 6.06530660e-01 1.35335283e-01
 1.11089965e-02 3.35462628e-04 3.72665317e-06]
b [[ 1. -5. 25.]
 [ 1. -4. 16.]
 [ 1. -3.  9.]
 [ 1. -2.  4.]
 [ 1. -1.  1.]
 [ 1.  0.  0.]
 [ 1.  1.  1.]
 [ 1.  2.  4.]
 [ 1.  3.  9.]
 [ 1.  4. 16.]
 [ 1.  5. 25.]]
ab [[ 3.72665317e-06 -1.86332659e-05  9.31663293e-05]
 [ 3.35462628e-04 -1.34185051e-03  5.36740205e-03]
 [ 1.11089965e-02 -3.33269896e-02  9.99809688e-02]
 [ 1.35335283e-01 -2.70670566e-01  5.41341133e-01]
 [ 6.06530660e-01 -6.06530660e-01  6.06530660e-01]
 [ 1.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 6.06530660e-01  6.06530660e-01  6.06530660e-01]
 [ 1.35335283e-01  2.70670566e-01  5.41341133e-01]
 [ 1.11089965e-02  3.33269896e-02  9.99809688e-02]
 [ 3.35462628e-04  1.34185051e-03  5.36740205e-03]
 [ 3.72665317e-06  1.86332659e-05  9.31663293e-05]]
abb [[[ 3.72665317e-06 -1.86332659e-05  9.31663293e-05]
  

TypeError: 'tuple' object cannot be interpreted as an integer

In [31]:
5*3.72665317e-06

1.863326585e-05